In [1]:
from time import time
start = time()

In [2]:
import os
GPUs = ','.join([str(i) for i in range(0,4)])
num_gpus = len(GPUs.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = GPUs

In [3]:
import dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess
import xgboost as xgb
import gc
    
def get_ip():
    cmd = "hostname --all-ip-addresses"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    IPADDR = str(output.decode()).split()[0]
    print(IPADDR)
    return IPADDR

def get_cluster():
    ip = get_ip()
    cluster = LocalCUDACluster(ip=ip,
                               device_memory_limit='10GB', 
                               jit_unspill=True,
                               protocol="ucx", 
                               rmm_pool_size="29GB")
    client = Client(cluster)
    return client


In [4]:
client = get_cluster()
client

10.120.104.12


2023-04-13 18:40:48,413 - distributed.comm.ucx - WARNING - A CUDA context for device 0 (b'GPU-67be321d-9012-d150-fdc9-402004163697') already exists on process ID 16542. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
2023-04-13 18:40:49,797 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space-10093/worker-wedbxe9e', purging
2023-04-13 18:40:49,797 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space-10093/worker-fhhxi8lo', purging
2023-04-13 18:40:49,798 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 18:40:49,798 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 18:40:49,798 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
202

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://10.120.104.12:8787/status,
Dashboard: http://10.120.104.12:8787/status,Workers: 4
Total threads: 4,Total memory: 1.48 TiB
Status: running,Using processes: True
Comm: ucx://10.120.104.12:50739,Workers: 4
Dashboard: http://10.120.104.12:8787/status,Total threads: 4
Started: Just now,Total memory: 1.48 TiB
Comm: ucx://10.120.104.12:46217,Total threads: 1
Dashboard: http://10.120.104.12:35915/status,Memory: 377.64 GiB
Nanny: ucx://10.120.104.12:47309,


In [5]:
%%time

users = dask_cudf.read_parquet('/raid/otto/Otto-Comp/pqs/train_v152_*.pq')

CPU times: user 951 ms, sys: 513 ms, total: 1.46 s
Wall time: 1.38 s


In [6]:
VER = 186
USE = 'clicks'

In [7]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [8]:
%%time

dtrain = xgb.dask.DaskQuantileDMatrix(client, users[FEATURES], users['clicks'])
del users

CPU times: user 3.17 s, sys: 431 ms, total: 3.6 s
Wall time: 28.9 s


In [9]:
gc.collect()

212

In [10]:
FOLDS = 5
SEED = 42

LR = 0.1

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':LR, 
    'subsample':0.7,
    'colsample_bytree':0.5, 
    'eval_metric':'map',
    'objective':'binary:logistic',
    'scale_pos_weight':8,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [11]:
%%time

output = xgb.dask.train(
        client,
        xgb_parms,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train")],
    )

/raid/data/mambaforge/envs/rapids-23.04/lib/python3.10/site-packages/xgboost/dask.py:856: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
INFO:distributed.scheduler:Receive client connection: Client-worker-7896770a-da65-11ed-81bc-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:50739
INFO:distributed.scheduler:Receive client connection: Client-worker-789671aa-da65-11ed-81bf-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:50739
INFO:distributed.scheduler:Receive client connection: Client-worker-78967094-da65-11ed-81c1-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:50739
INFO:distributed.scheduler:Receive client connection: Client-worker-78abab68-da65-11ed-81b9-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:50739
[18:41:27] task [xgboost.dask-0]:ucx://10.120.104.

[0]	train-map:0.16598
[1]	train-map:0.17580
[2]	train-map:0.17586
[3]	train-map:0.17439
[4]	train-map:0.17784
[5]	train-map:0.17831
[6]	train-map:0.17851
[7]	train-map:0.18267
[8]	train-map:0.18142
[9]	train-map:0.18237
[10]	train-map:0.18633
[11]	train-map:0.18673
[12]	train-map:0.18650
[13]	train-map:0.18695
[14]	train-map:0.18747
[15]	train-map:0.18766
[16]	train-map:0.18776
[17]	train-map:0.18807
[18]	train-map:0.18823
[19]	train-map:0.18845
[20]	train-map:0.18910
[21]	train-map:0.18930
[22]	train-map:0.18973
[23]	train-map:0.18993
[24]	train-map:0.18995
[25]	train-map:0.19023
[26]	train-map:0.19023
[27]	train-map:0.19032
[28]	train-map:0.19053
[29]	train-map:0.19078
[30]	train-map:0.19110
[31]	train-map:0.19147
[32]	train-map:0.19190
[33]	train-map:0.19248
[34]	train-map:0.19359
[35]	train-map:0.19463
[36]	train-map:0.19486
[37]	train-map:0.19551
[38]	train-map:0.19535
[39]	train-map:0.19565
[40]	train-map:0.19601
[41]	train-map:0.19631
[42]	train-map:0.19676
[43]	train-map:0.1969

In [12]:
duration = time()-start
duration

217.7893419265747